In [1]:
import numpy as np
import math
import torch
import torch.nn as nn
from utils import get_train_test, setup_seed

seed=0
setup_seed(seed)

K = 4
Rated_Capacity = 1.1
feature_size = 6
feature_num = K
dropout = 0.0
epochs = 500
nhead = 1
hidden_dim = 16
num_layers = 3
lr = 0.001  
weight_decay = 0.0
noise_level = 0.0
alpha = 0.1
metric = 're'
seed = 0
device = "cpu"

# Load Data

In [2]:
Battery = np.load('datasets/CALCE/CALCE.npy', allow_pickle=True)
Battery = Battery.item() # A dict with cell name as key and dataframe as value
name = 'CS2_35'
train_x, train_y, train_data, test_data = get_train_test(Battery, name, feature_size)
print(f'train_x.shape: {train_x.shape}, train_y.shape: {train_y.shape}')

train_x.shape: (2886, 6), train_y.shape: (2886,)


In [3]:
x = np.reshape(train_x/Rated_Capacity,(-1, 1, feature_size)).astype(np.float32)
y = np.reshape(train_y/Rated_Capacity,(-1,1)).astype(np.float32) 
print(f'after reshape\nx.shape: {x.shape}\ny.shape: {y.shape}')

x, y = torch.from_numpy(x).to(device), torch.from_numpy(y).to(device)
x = x.repeat(1, K, 1)
print(f'after repeat\nx.shape: {x.shape}\ny.shape: {y.shape}')

after reshape
x.shape: (2886, 1, 6)
y.shape: (2886, 1)
after repeat
x.shape: torch.Size([2886, 4, 6])
y.shape: torch.Size([2886, 1])


In [13]:
class PositionalEncoding(nn.Module):
    def __init__(self, feature_len, feature_size, dropout=0.0):
        # feature_len: max length of sequence, feature_size: embedding size(d_model)
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(feature_len, feature_size)
        position = torch.arange(0, feature_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, feature_size, 2).float() * (-math.log(10000.0) / feature_size))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        # print(f'pe.shape: {pe.shape}')
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe

  
class Transformer(nn.Module):
    def __init__(self, feature_size, hidden_dim, feature_num, num_layers, nhead):
        super(Transformer, self).__init__()
        half_feature_size = int(feature_size / 2)
        print(f'dropout: {dropout}')
        
        self.autoencoder = nn.Linear(feature_size, half_feature_size)
        self.pe = PositionalEncoding(half_feature_size, feature_num)
        encoder_layer = nn.TransformerEncoderLayer(d_model=feature_num, nhead=nhead, dim_feedforward=hidden_dim, dropout=dropout, batch_first=True)
        self.layers = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.lm_head = nn.Linear(feature_num*half_feature_size, 1)
        
    def forward(self, x):
        B,T,C = x.shape
        print(f'input x.shape: {x.shape}')
        print(x)
        x = self.autoencoder(x)
        print(f'after autoencoder, x.shape: {x.shape}')
        print(x)
        x = x.reshape(B, -1, T)
        print(f'after reshape before positional encoding, x.shape: {x.shape}')
        print(x)
        x = self.pe(x)
        print(f'after positional encoding, x.shape: {x.shape}')
        print(x)
        x = self.layers(x)
        print(f'after transformer layers, x.shape: {x.shape}')
        # print(x)
        x = x.reshape(B, -1)
        print(f'after reshape before lm_head, x.shape: {x.shape}')
        # print(x)
        x = self.lm_head(x)
        print(f'after lm_head, x.shape: {x.shape}')
        # print(x)
        return x

In [14]:
model = Transformer(feature_size, hidden_dim, feature_num, num_layers, nhead)

dropout: 0.0


In [15]:
out = model(x)

input x.shape: torch.Size([2886, 4, 6])
tensor([[[1.0307, 1.0304, 1.0298, 1.0246, 1.0243, 1.0221],
         [1.0307, 1.0304, 1.0298, 1.0246, 1.0243, 1.0221],
         [1.0307, 1.0304, 1.0298, 1.0246, 1.0243, 1.0221],
         [1.0307, 1.0304, 1.0298, 1.0246, 1.0243, 1.0221]],

        [[1.0304, 1.0298, 1.0246, 1.0243, 1.0221, 1.0136],
         [1.0304, 1.0298, 1.0246, 1.0243, 1.0221, 1.0136],
         [1.0304, 1.0298, 1.0246, 1.0243, 1.0221, 1.0136],
         [1.0304, 1.0298, 1.0246, 1.0243, 1.0221, 1.0136]],

        [[1.0298, 1.0246, 1.0243, 1.0221, 1.0136, 1.0082],
         [1.0298, 1.0246, 1.0243, 1.0221, 1.0136, 1.0082],
         [1.0298, 1.0246, 1.0243, 1.0221, 1.0136, 1.0082],
         [1.0298, 1.0246, 1.0243, 1.0221, 1.0136, 1.0082]],

        ...,

        [[0.3500, 0.3417, 0.3417, 0.3333, 0.3333, 0.3333],
         [0.3500, 0.3417, 0.3417, 0.3333, 0.3333, 0.3333],
         [0.3500, 0.3417, 0.3417, 0.3333, 0.3333, 0.3333],
         [0.3500, 0.3417, 0.3417, 0.3333, 0.3333, 0.333

- Get data [B,T] from raw data
- Repeat T for k time and get [B, K, T]
- Linear Map T to T/2 get [B, K, T/2]
  - There is no sequence info in T/2?
- reshape [not transpose] to [B, T/2, K] and then pass to Transformer layer
  - The sequence now doesn't make any sense
- go through lm_head, return one value at dimension T